In [19]:
%run ../src/game.py
%run ../src/ipd.py
%run ../src/strategies.py
%run ../src/tools.py
dip =[(3,3),(0,5),(5,0),(1,1)]   # Dilemme du prisonnier
g = Game(dip,['C','D'])


# Evaluation par synthèse de sous-classes

Les compétitions écologiques offrent un outil de mesure assez fiable de la robustesse d'une stratégies, mais encore insuffisant. Il se peut par exemple que certaines stratégies se sacrifient pour d'autres dans un schema *maitre-esclave*. Avoir une synthèse de centaines voir de milliers de compétitions écologiques dans lesquelles on a enlevé certaines stratégies mesure sans doute une meilleure robustesse. L'une des idées les plus simples consiste à calculer les n compétitions possibles que l'on peut faire en enlevant 1 stratégie à un ensemble de n stratégies. On appelle cette technique la technique des sous-classes.
Nous définissions ici 3 fonctions permettant de réaliser ces sous-classes.
- `subclasses(bag, n)` qui évalue tous les sous ensembles possibles de taille n dans la bage
- `subclassesWithOneStrat(bag, n, strat)` qui évalue Strat dans tous les sous ensembles possibles de taille n dans la bage en ajoutant systématiquement la stratégie strat
- `subclassesRandomWithOneStrat(p, bag, n, Strat)` qui réalise p competitions de n strategies choisies au hasard dans la bage dans lesquelles on rajoute systématiquement Strat

Les évaluations réalisées dans ces fonctions sont des compétitions écologiques.

Ces fonctions renvoient à la fin un tableau avec pour chaque stratégie, sa meilleure place, sa pire place, sa moyenne et son écart-type.

#### Un cas simple : toutes les compétitions de 3 stratégies parmi les classiques

In [12]:
All_C = Periodic('C')
All_D = Periodic('D')
bag = [All_C, All_D, Tft(), Spiteful(), Gradual(), SoftMajority(), HardMajority()]
subClasses(bag, 3)

          BestRank  WorstRank   RankAvg   RankStd
tft            1.0        2.0  1.133333  0.351866
softmajo       1.0        2.0  1.133333  0.351866
spiteful       1.0        2.0  1.266667  0.457738
gradual        1.0        2.0  1.333333  0.487950
per_C          1.0        3.0  1.400000  0.632456
per_D          1.0        3.0  2.533333  0.743223
hardmajo       2.0        3.0  2.666667  0.487950


#### Un cas plus volumineux : toutes les stratégies Mem(1,1) avec une stratégie en moins à chaque fois
Sachant qu'il y a 32 `mem(1,1)` cette opération réalise donc 32 compétitions de 31 stratégie. Notez que dans le cas de cette méthode, toutes les stratégies sont présentes (et absentes) exactement le même nombre de fois.


In [ ]:
bag = getAllMemory(1,1)
subClasses(bag, len(bag)-1)
after = time.time()

#### Test de la stratégie Spiteful avec tous les triplets de classiques
Dans les cas des deux méthodes `subClassesWithOneStrat` seule la stratégie passée en paramètre participe à toutes les sous-classes (faisable pour des ensembles pas trop volumineux genre `mem(1,1)`). Dans la première elle participe à la totalité des sous-classes tandis qu'avec `subClassesRandomWithOneStrat` elle participe à un nombre fixé de sous classes de même taille mais prises aléatoirement (utilisable dans de gros ensembles comme `mem(2,2)`).

In [ ]:
All_C = Periodic('C')
All_D = Periodic('D')
bag = [All_C, All_D, Tft(), Gradual(), SoftMajority(), HardMajority()]
res  = subClassesWithOneStrat(bag, 3, Spiteful())
#Pour afficher le tableau en entier : 
#res = subClassesWithOneStrat(bag, 3, Spiteful(), True)

Il est à noter que les objets `subClassesWithOneStrat` et `subClassesRandomWithOneStrat` conservent le 
meilleur et le pire des tournois pour la stratégie `strat`
Au moment de l'affichage du classement des sous-classes il est de ce fait possible d'afficher l'ensemble de stratégies qui a été favorable ou défavorable à la stratégie `Strat`.

In [ ]:
meilleureComp, pireComp, strategy = res
print("La meilleure competition pour la stratégie "+strategy.name +" est : ")
for strat in meilleureComp :
    print(strat.name)

#### 100 experiences de 10 stratégies prise au hasard dans mem(2,2) contre Gradual()
Pour les `subclassesRandom`, si une stratégie n'a joué qu'une seule fois alors elle n'a pas d'écart-type (`NaN`); si elle n'a pas joué du tout alors toutes ses valeurs sont à `NaN` dans le tableau

In [ ]:
bag = getAllMemory(2,2)
res = subClassesRandomWithOneStrat(100,bag, 10, Gradual())
#Pour afficher le tableau en entier : 
#subClassesRandomWithOneStrat(100, bag, 10, Gradual(), True)

# Attention : cette expérience prend de l'ordre de 1mn

On peut d'ailleurs vérifier la compétition la plus "défavorable" à Gradual 
(comme c'est un choix aléatoire, donc 2 éxécutions ne donneront pas systématiquement le même résultat)

In [ ]:
bestComp, worstComp, strategy = res

In [ ]:
bag = worstComp
e2=Ecological(g,bag)
e2.run()
e2.drawPlot(None,None)
evol=e2.historic
print(evol.iloc[-1])
print(e2.historic.iloc[e2.generation].rank(0, method="min", ascending=False))

## Tests de robustesse de MetaStratégie

In [13]:
metaStrat = MetaStrategy([Tft(), Periodic("C"), Spiteful(), Periodic("CCD")], 5)
bag = getMem(1,1)+[Gradual()]
res  = subClassesWithOneStrat(bag,len(bag)-1, metaStrat)

Strategy ranking  : metastrat
BestRank     2.000000
WorstRank    6.000000
RankAvg      5.757576
RankStd      0.751262
Name: metastrat, dtype: float64


In [84]:
metaStrat = MetaStrategy([Tft(), Spiteful(), Prober(), Gradual()], 5)
bag = getMem(1,1)+[Gradual()]
res  = subClassesWithOneStrat(bag,len(bag)-1, metaStrat)

Strategy ranking  : metastrat
BestRank     2.000000
WorstRank    3.000000
RankAvg      2.939394
RankStd      0.242306
Name: metastrat, dtype: float64


In [87]:
metaStrat = MetaStrategy([Tft(), Spiteful(), Gradual()], 5)
bag = getMem(1,1)+[Gradual()]
res  = subClassesWithOneStrat(bag,len(bag)-1, metaStrat)

Strategy ranking  : metastrat
BestRank     2.000000
WorstRank    3.000000
RankAvg      2.939394
RankStd      0.242306
Name: metastrat, dtype: float64


## Tests d'équivalence de stratégies

`equivalence` est une fonction permettant d'évaluer si une liste de stratégies ont le même comportement face à une soupe de stratégies (en Tournoi). Elle renvoie True si toutes les stratégies sont équivalentes et False sinon.

In [76]:
def equivalence(l, soup, length):
    res = []
    for strat in l :
        t = Tournament(g, soup+[strat], length)
        t.run()
        s = t.matrix[strat.name].to_string()
        res += [tuple([int(i) for i in s.split() if i.isdigit()])] 
    return res, len(set(res)) < len(l)
        
        
equivalence([Periodic("C"), Periodic("C")], [Periodic("D"), Tft()], 10)
equivalence([Periodic("CCCD"), Periodic("CCDCD")], [Periodic("D")],2)

([(10, 6), (10, 6)], True)

In [93]:
def simplify(soup, opponents , length):
    res, b = equivalence(soup, opponents, length)
    if b : 
        s = set(res)
        s = list(s)
        ind = []
        for i in range(len(s)) :
            ind += [res.index(s[i])]
            
        simplified = []
        for i in range(len(ind)):
            simplified += [soup[ind[i]]]
        
        print("{} strategies deleted. ".format(len(res)-len(simplified)))
        return len(simplified), simplified
    else : 
        return soup
    
simplify([Periodic("CCCD"), Periodic("CCDCD"), Periodic('D')], getMem(1,1), 2)
simplify(getMem(1,1), [Periodic('C'), Periodic('D')], 10)
        

1 strategies deleted. 
11 strategies deleted. 


(21,
  <__main__.Mem at 0x7fecd6403898>])

In [96]:
res12 = simplify(getMem(1,2), [Periodic('C'), Periodic('D')], 10)
res21 = simplify(getMem(2,1), [Periodic('C'), Periodic('D')], 10)
#res22 = simplify(getMem(2,2), [Periodic('C'), Periodic('D')], 10)

tab = pd.DataFrame(
        np.nan, ["Mem 1 1", "Mem 1 2", "Mem 2 1", "Mem 2 2"], ["All strategies", "After simplify"]
    )
tab.at["Mem 1 1", "All strategies" ] = 32
tab.at["Mem 1 2", "All strategies" ] = 1024
tab.at["Mem 2 1", "All strategies" ] = 1024
tab.at["Mem 2 2", "All strategies" ] = 2048
tab.at["Mem 1 1", "After simplify" ] = 21
tab.at["Mem 1 2", "After simplify" ] = res12[0]
tab.at["Mem 2 1", "After simplify" ] = res21[0]
#tab.at["Mem 2 2", "After simplify" ] = res22[0]

tab


925 strategies deleted. 
843 strategies deleted. 


,All strategies,After simplify
Mem 1 1,32.0,21.0
Mem 1 2,1024.0,99.0
Mem 2 1,1024.0,181.0
Mem 2 2,2048.0,NaN
